# Project - Building a Spam Filter with Naive Bayes

To classify messages as spam or non-spam, we saw in the previous mission that the computer:

1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
3. Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

In [27]:
import pandas as pd
df = pd.read_csv("E:\\DataQuest\\Projects\\Naive Bayes\\SMS-Spam Filter\\SMS-Spam-Filter-main\\SMSSpamCollection",sep='\t',header=None,names = ['Label','SMS'])

In [28]:
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [29]:
df['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

So, approximately 87% of messages are not spam (is ham) while about 13% of messages are spam based on the classification done by humans in the provided dataset.

## Training and Test set

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

1. A training set, which we'll use to "train" the computer how to classify messages.
2. A test set, which we'll use to test how good the spam filter is with classifying new messages.

-----------------------------------------------------------------------------------------------

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

1. The training set will have 4,458 messages (about 80% of the dataset).
2. The test set will have 1,114 messages (about 20% of the dataset).



In [30]:
randomized_data = df.sample(frac=1,random_state=1)

# getting index for split
training_tst_index = round(len(randomized_data)*0.8)
training_tst_index # 4458

#splliting of data
train = randomized_data[:training_tst_index].reset_index(drop=True)
train.head()
test = randomized_data[training_tst_index:].reset_index(drop=True)
test.head()

print(train.shape)
print(test.shape)

(4458, 2)
(1114, 2)


In [31]:
train['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [32]:
test['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

# Letter Case and Punctuation

## performing bit data cleaning initially

In [33]:
train.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [34]:
import re
train['SMS'] = train['SMS'].str.replace('\W',' ')
train['SMS'] = train['SMS'].str.lower()

<ipython-input-34-f318e0b5b31b>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['SMS'] = train['SMS'].str.replace('\W',' ')


In [35]:
print(train['Label'].value_counts(normalize=True))
train.head()

ham     0.86541
spam    0.13459
Name: Label, dtype: float64


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


# Creating a Vocabulary

In [36]:
train['SMS'] = train['SMS'].str.split()
#print(train['SMS'])
vocabulary = []
for msg in train['SMS']:
    for word in msg:
        vocabulary.append(word)

vocabulary = list(set(vocabulary)) # list made into set using set function to remove duplicate vocabs then converted back to list using list function
len(vocabulary)

7783

# The Final Training Set

To create the dictionary we need for our training set, we can use the code below, where:

1.) We start by initializing a dictionary named word_counts_per_sms, where each key is a unique word (a string) from the vocabulary, and each value is a list of the length of training set, where each element in the list is a 0.
1.1) The code [0] * 5 outputs [0, 0, 0, 0, 0]. So the code [0] * len(training_set['SMS']) outputs a list of the length of training_set['SMS'], where each element in the list will be a 0.

2.) We loop over training_set['SMS'] using at the same time the enumerate() function to get both the index and the SMS message (index and sms).
2.1.) Using a nested loop, we loop over sms (where sms is a list of strings, where each string represents a word in a message).
            We incremenent word_counts_per_sms[word][index] by 1.


In [37]:
words_count_per_msg = {unique_word : [0] * len(train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train['SMS']):
    for word in sms:
        words_count_per_msg[word][index] += 1
print(len(words_count_per_msg))

7783


In [38]:
new_df = pd.DataFrame(words_count_per_msg)
print(new_df.shape)
new_df.head()

(4458, 7783)


,kitty,queries,studyn,mobsi,sing,seat,potato,elvis,nitw,frying,...,boggy,dokey,100p,heltini,fakeye,lololo,outfor,smsservices,five,type
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# Concatenating the dataframes
clean_train = pd.concat([train,new_df],axis=1)
print(clean_train.shape)

(4458, 7785)


In [40]:
clean_train.head()

,Label,SMS,kitty,queries,studyn,mobsi,sing,seat,potato,elvis,...,boggy,dokey,100p,heltini,fakeye,lololo,outfor,smsservices,five,type
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Calculating Constants First

## P(Spam|w1,w2,...,wn) ∝ P(Spam) ⋅ n∏i=1P(wi|Spam)
## P(Ham|w1,w2,...,wn) ∝ P(Ham) ⋅ n∏i=1P(wi|Ham)

In [41]:
# Using Training Data only
print(clean_train['Label'].value_counts())
clean_train.head()

ham     3858
spam     600
Name: Label, dtype: int64


,Label,SMS,kitty,queries,studyn,mobsi,sing,seat,potato,elvis,...,boggy,dokey,100p,heltini,fakeye,lololo,outfor,smsservices,five,type
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# seperating data based on labels
ham_msg = clean_train[clean_train['Label']=='ham'] # 3858
spam_msg = clean_train[clean_train['Label']=='spam'] # 600
# print(clean_train.shape)
# print(ham_msg.shape)
# print(spam_msg.shape)
# print('train : ',clean_train.shape)
# print(len(clean_train)) # above line of code
# print(train.shape)
# Computing p_spam
p_ham = ham_msg.shape[0]/clean_train.shape[0] # 0.8654104979811574
p_spam = spam_msg.shape[0]/clean_train.shape[0] #0.13458950201884254

# Computing N_ham
n_per_msg_ham = ham_msg['SMS'].apply(len) # apply returns the series of applied function's output
N_ham = n_per_msg_ham.sum()
# print(N_ham)

# Computing N_spam
n_per_msg_spam = spam_msg['SMS'].apply(len)
N_spam = n_per_msg_spam.sum()
print(N_spam)

# Computing N_vocab
N_vocab = len(vocabulary)
print("vocab : ",N_vocab)
# Additive Smoothing parameter-α (Laplace-Smoothing)
alpha = 1

# print(p_spam)
# print(p_ham)

15190
vocab :  7783


In [43]:
spam_msg.head()

,Label,SMS,kitty,queries,studyn,mobsi,sing,seat,potato,elvis,...,boggy,dokey,100p,heltini,fakeye,lololo,outfor,smsservices,five,type
16,spam,"[freemsg, why, haven, t, you, replied, to, my,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,spam,"[congrats, 2, mobile, 3g, videophones, r, your...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,spam,"[free, message, activate, your, 500, free, tex...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,spam,"[call, from, 08702490080, tells, u, 2, call, 0...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,spam,"[someone, has, conacted, our, dating, service,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Calculating Parameters


1. P(Spam) and P(Ham)
2. NSpam, NHam, NVocabulary

As we've already mentioned, all these terms will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(wi|Spam) and P(wi|Ham) vary depending on the word, the probability for each individual word is constant for every new message.

For instance, let's say we receive two new messages:

    "secret code"
    "secret party 2night"

We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values we need to find a result for the equation below:

P("secret"|Spam)=N"secret"|Spam+αNSpam+α⋅NVocabulary

The steps we take to calculate P("secret"|Spam) will be identical for both of our new messages above, or for any other new message that contains the word "secret". The key detail here is that calculating P("secret"|Spam) only depends on the training set, and as long as we don't make changes to the training set, P("secret"|Spam) stays constant. The same reasoning also applies to P("secret"|Ham).

This means that we can use our training set to calculate the probability for each word in our vocabulary. If our vocabulary contained only the words "lost", "navigate", and "sea", then we'd need to calculate six probabilities:

    P("lost"|Spam) and P("lost"|Ham)
    P("navigate"|Spam) and P("navigate"|Ham)
    P("sea"|Spam) and P("sea"|Ham)

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called "Parameters".

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message. 

Also, if these values are not calculated initially then there will be repeatation of calculations that will just slow/drag down the algorithm.

Parameters to compute : 

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [44]:
# ham_msg & spam_msg are 2 dataframes made in previous cell
# inititalizing 2 dictionaries each for spam and ham words
ham_parameters = {unique_words : [0] for unique_words in vocabulary}
spam_parameters = {unique_words : [0] for unique_words in vocabulary}


# Isolating spam and ham data in different dataframes
# spam_data = clean_train[clean_train['Label']=='spam']
# ham_data = clean_train[clean_train['Label']=='ham']
# print()
# iterating over vocabulary for spam
for w in vocabulary:
    n_w_given_spam = spam_msg[w].sum() # spam_msg in previous cell
    p_w_given_spam = (n_w_given_spam+alpha)/(N_spam + alpha*N_vocab) # N_vocab in previous cell
    spam_parameters[w] = p_w_given_spam
    

    # Similarly,
    n_w_given_ham = ham_msg[w].sum()
    p_w_given_ham = (n_w_given_ham+alpha)/(N_ham + alpha*N_vocab)
    ham_parameters[w] = p_w_given_ham

print('ham_parameters len : ',len(ham_parameters))
print('spam_parameters len : ',len(spam_parameters))


ham_parameters len :  7783
spam_parameters len :  7783


In [45]:
# print(spam_parameters)
# print(spam_parameters['greatly'])# 0.00012848515996402416

# Classifying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

1. Takes in as input a new message (w1, w2, ..., wn)
2. Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
3. Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:

        If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
        If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
        If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.


In [46]:
import re
def classify(message):
    message = re.sub('\W',' ',message)
    message = message.lower()
    message = message.split()

    '''
    Computing p_spam_given_msg & p_ham_given_msg both below
    '''
    p_spam_given_msg = p_spam
    p_ham_given_msg = p_ham 

    for w in message:
        if w in spam_parameters:
            p_spam_given_msg *= spam_parameters[w]
        if w in ham_parameters:
            p_ham_given_msg *= ham_parameters[w]
        #else:
        #    continue
    print('P(Spam|msg) = ',p_spam_given_msg)
    print('P(Ham|msg) = ',p_ham_given_msg)
    
    if p_ham_given_msg > p_spam_given_msg:
        print('Label: Ham')
    elif p_ham_given_msg < p_spam_given_msg:
        print('Label: Spam')
    else:
        print('Equal probabiliies, have a human classify this!')

In [47]:
# Testing function defined above
m1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
classify(m1)
print('------------------------------------------------------')
m2 = "Sounds good, Tom, then see u there"
classify(m2)

P(Spam|msg) =  1.3481290211300841e-25
P(Ham|msg) =  1.9368049028589875e-27
Label: Spam
------------------------------------------------------
P(Spam|msg) =  2.4372375665888117e-25
P(Ham|msg) =  3.687530435009238e-21
Label: Ham


# Measuring Spam Filter's Accuracy

Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:
-------------------------------------------------------------------------------------------
FORMULA:


Accuracy = (number of correctly classified messages) / (total number of classified messages)

In [48]:
total = test.shape[0]

In [49]:
# Updating classify function for test-data
import re
def classify_test(message):
    message = re.sub('\W',' ',message)
    message = message.lower()
    message = message.split()

    '''
    Computing p_spam_given_msg & p_ham_given_msg both below
    '''
    p_spam_given_msg = p_spam
    p_ham_given_msg = p_ham 

    for w in message:
        if w in spam_parameters:
            p_spam_given_msg *= spam_parameters[w]
        if w in ham_parameters:
            p_ham_given_msg *= ham_parameters[w]
        #else:
        #    continue
    
    if p_ham_given_msg > p_spam_given_msg:
        return 'ham'
    elif p_ham_given_msg < p_spam_given_msg:
        return 'spam'
    else:
        return 'needs human classification'

In [50]:
'''
we have a function that returns labels instead of printing them, we can use it to create a new column in our test set
'''
test['predicted'] = test['SMS'].apply(classify_test)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [51]:
correct = 0
for row in test.iterrows():
    row = row[1] # to avoid error of tuple as if this is not done row is in tuple format
    if row['Label'] == row['predicted']:
        correct += 1

print('Correct = ',correct)
print(total)
print('Incorrect = ',total-correct)
print('Accuracy = ',(correct/total)*100)

Correct =  1100
1114
Incorrect =  14
Accuracy =  98.74326750448833


The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.

h
